In [1]:
import os

In [2]:
%pwd

'F:\\Study\\Deep_learning_Project\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'F:\\Study\\Deep_learning_Project\\Kidney-Disease-Classification'

In [7]:
#entitiy
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [12]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_ymal, create_directories

In [39]:
#configuration
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_ymal(config_filepath)
        self.params = read_ymal(params_filepath)
        create_directories([self.config.artifact_root])

    def data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file =  config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        

In [16]:
import os
import zipfile
from src.cnnClassifier import logger
import gdown
from src.cnnClassifier.utils.common import get_size

In [43]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self) -> str:
        """Fetch data from the url"""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")

            field_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+field_id,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the zip file into data directory
        returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [44]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-16 22:15:46,640: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-16 22:15:46,644: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-16 22:15:46,646: INFO: common: created directory at: artifacts]
[2024-03-16 22:15:46,649: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-16 22:15:46,651: INFO: 2045771997: Downloading data from https://drive.google.com/file/d/1AH7xXZfheFP6VXleI-Rbpvf8rR5R95-6/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1AH7xXZfheFP6VXleI-Rbpvf8rR5R95-6
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1AH7xXZfheFP6VXleI-Rbpvf8rR5R95-6&confirm=t&uuid=af4543cd-0e26-49b8-8ade-dadcb7be4662
To: F:\Study\Deep_learning_Project\Kidney-Disease-Classification\artifacts\data_ingestion\data.zip
100%|███████████████████████████████████████████████████████████████████████████████| 248M/248M [00:41<00:00, 5.99MB/s]

[2024-03-16 22:16:33,121: INFO: 2045771997: Downloaded data from https://drive.google.com/file/d/1AH7xXZfheFP6VXleI-Rbpvf8rR5R95-6/view?usp=sharing into file artifacts/data_ingestion/data.zip]


ConfigBox({'artifact_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1AH7xXZfheFP6VXleI-Rbpvf8rR5R95-6/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

'artifacts'